In [3]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import ResNet50, MobileNetV2, EfficientNetB0, DenseNet121
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


In [4]:
# Update the dataset path as per Kaggle environment
DATASET_PATH = "/kaggle/input/ebhi-seg/EBHI-SEG"  

# Define all categories present in the dataset
CATEGORIES = ['Adenocarcinoma', 'High-grade IN', 'Low-grade IN', 'Normal', 'Polyp', 'Serrated adenoma']

IMG_HEIGHT, IMG_WIDTH = 224, 224  # Standard input size for deep learning models
NUM_CLASSES = 2  # Binary Segmentation (Cancer vs. Non-Cancer)


In [5]:
def load_dataset(dataset_path, categories):
    images, masks = [], []

    for category in categories:
        image_dir = os.path.join(dataset_path, category, "image")
        mask_dir = os.path.join(dataset_path, category, "label")

        image_files = sorted(os.listdir(image_dir))
        mask_files = sorted(os.listdir(mask_dir))

        for img_name, mask_name in zip(image_files, mask_files):
            img_path = os.path.join(image_dir, img_name)
            mask_path = os.path.join(mask_dir, mask_name)

            img = load_img(img_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
            mask = load_img(mask_path, target_size=(IMG_HEIGHT, IMG_WIDTH), color_mode="grayscale")

            img = img_to_array(img) / 255.0  # Normalize image
            mask = img_to_array(mask) / 255.0  # Normalize mask
            mask = np.where(mask > 0.5, 1, 0)  # Convert to binary mask

            images.append(img)
            masks.append(mask)

    return np.array(images), np.array(masks)

images, masks = load_dataset(DATASET_PATH, CATEGORIES)



In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(images, masks, test_size=0.2, random_state=42)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.1, random_state=42)

print("Training:", X_train.shape, Y_train.shape)
print("Validation:", X_val.shape, Y_val.shape)
print("Testing:", X_test.shape, Y_test.shape)



Training: (1602, 224, 224, 3) (1602, 224, 224, 1)
Validation: (178, 224, 224, 3) (178, 224, 224, 1)
Testing: (446, 224, 224, 3) (446, 224, 224, 1)


In [7]:
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.RandomContrast(0.1),
])


In [8]:
import os

AUGMENTED_PATH = "/kaggle/working/augmented_ebhi_seg"  # Define a path to save augmented data

# Create the main directory if not exists
os.makedirs(AUGMENTED_PATH, exist_ok=True)

# Create subdirectories for each category
for category in CATEGORIES:
    os.makedirs(os.path.join(AUGMENTED_PATH, category, "images"), exist_ok=True)
    os.makedirs(os.path.join(AUGMENTED_PATH, category, "masks"), exist_ok=True)

print("Folders created successfully!")


Folders created successfully!


In [9]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img

# Define augmentation techniques
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode="nearest"
)

def augment_and_save(images, masks, category, num_augments=2):
    """
    Perform data augmentation and save images and masks to new files.

    Parameters:
        images (numpy array): Input images (batch_size, 224, 224, 3)
        masks (numpy array): Corresponding masks (batch_size, 224, 224, 1)
        category (str): Category name (e.g., 'Adenocarcinoma')
        num_augments (int): Number of augmented images to generate per image
    """
    for i in range(len(images)):
        img = images[i]
        mask = masks[i]
        
        # Expand dims to create batch dimension (1, 224, 224, 3) and (1, 224, 224, 1)
        img = np.expand_dims(img, axis=0)  
        mask = np.expand_dims(mask, axis=0)  

        # Ensure mask dtype is compatible (convert float64 to uint8)
        mask = mask.astype(np.uint8)

        img_gen = datagen.flow(img, batch_size=1, save_prefix="image", save_format="png")
        mask_gen = datagen.flow(mask, batch_size=1, save_prefix="mask", save_format="png")

        for j in range(num_augments):
            aug_img = next(img_gen)[0]  # Convert to 3D
            aug_mask = next(mask_gen)[0]  # Convert to 3D

            # Ensure correct save paths
            img_filename = os.path.join(AUGMENTED_PATH, category, "images", f"aug_{i}_{j}.png")
            mask_filename = os.path.join(AUGMENTED_PATH, category, "masks", f"aug_{i}_{j}.png")

            # Convert arrays to image format and save
            array_to_img(aug_img).save(img_filename)
            array_to_img(aug_mask).save(mask_filename)

        # Save original image and mask as well
        original_img_filename = os.path.join(AUGMENTED_PATH, category, "images", f"original_{i}.png")
        original_mask_filename = os.path.join(AUGMENTED_PATH, category, "masks", f"original_{i}.png")

        array_to_img(images[i]).save(original_img_filename)
        array_to_img(masks[i]).save(original_mask_filename)

    print(f"Augmented data saved for {category}")

        

In [10]:
for category in CATEGORIES:
    print(f"Processing category: {category}")
    
    # Load dataset for this specific category
    images, masks = load_dataset(DATASET_PATH, [category])
    
    # Augment and save
    augment_and_save(images, masks, category, num_augments=2)  # Generates 2 new images per original


Processing category: Adenocarcinoma
Augmented data saved for Adenocarcinoma
Processing category: High-grade IN
Augmented data saved for High-grade IN
Processing category: Low-grade IN
Augmented data saved for Low-grade IN
Processing category: Normal
Augmented data saved for Normal
Processing category: Polyp
Augmented data saved for Polyp
Processing category: Serrated adenoma
Augmented data saved for Serrated adenoma


In [11]:
import os

# List the number of saved images in a category
category_to_check = "Normal"

num_images = len(os.listdir(os.path.join(AUGMENTED_PATH, category_to_check, "images")))
num_masks = len(os.listdir(os.path.join(AUGMENTED_PATH, category_to_check, "masks")))

print(f"Saved {num_images} images and {num_masks} masks for category {category_to_check}")


Saved 228 images and 228 masks for category Normal


In [12]:
def build_unet(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3), num_classes=NUM_CLASSES):
    inputs = keras.Input(shape=input_shape)

    # Encoder
    conv1 = layers.Conv2D(64, (3, 3), activation="relu", padding="same")(inputs)
    conv1 = layers.Conv2D(64, (3, 3), activation="relu", padding="same")(conv1)
    pool1 = layers.MaxPooling2D((2, 2))(conv1)

    conv2 = layers.Conv2D(128, (3, 3), activation="relu", padding="same")(pool1)
    conv2 = layers.Conv2D(128, (3, 3), activation="relu", padding="same")(conv2)
    pool2 = layers.MaxPooling2D((2, 2))(conv2)

    conv3 = layers.Conv2D(256, (3, 3), activation="relu", padding="same")(pool2)
    conv3 = layers.Conv2D(256, (3, 3), activation="relu", padding="same")(conv3)
    pool3 = layers.MaxPooling2D((2, 2))(conv3)

    # Bottleneck
    conv4 = layers.Conv2D(512, (3, 3), activation="relu", padding="same")(pool3)
    conv4 = layers.Conv2D(512, (3, 3), activation="relu", padding="same")(conv4)

  # Decoder: FULLY UPSAMPLING BACK TO (224, 224, 1)
    up5 = layers.Conv2DTranspose(256, (3, 3), strides=(2, 2), padding="same")(conv4)  # (56, 56)
    up5 = layers.concatenate([up5, conv3])
    conv5 = layers.Conv2D(256, (3, 3), activation="relu", padding="same")(up5)

    up6 = layers.Conv2DTranspose(128, (3, 3), strides=(2, 2), padding="same")(conv5)  # (112, 112)
    up6 = layers.concatenate([up6, conv2])
    conv6 = layers.Conv2D(128, (3, 3), activation="relu", padding="same")(up6)

    up7 = layers.Conv2DTranspose(64, (3, 3), strides=(2, 2), padding="same")(conv6)  # (224, 224)
    up7 = layers.concatenate([up7, conv1])
    conv7 = layers.Conv2D(64, (3, 3), activation="relu", padding="same")(up7)

    outputs = layers.Conv2D(1, (1, 1), activation="sigmoid", padding="same")(conv7)  # Ensure (224, 224, 1)

    return keras.Model(inputs, outputs)

unet_model = build_unet()

In [13]:
def build_unet_backbone(base_model, input_shape=(224, 224, 3)):
    inputs = keras.Input(shape=input_shape)

    # Load base model WITHOUT top layers
    base = base_model(input_tensor=inputs, include_top=False, weights=None)  

    # Extract last feature map
    x = base.output  # Default shape is typically (7,7,C) or (14,14,C)

    # Decoder: Force Upsampling to (224, 224, 1)
    x = layers.Conv2DTranspose(512, (3, 3), strides=(2, 2), padding="same")(x)  # (14, 14)
    x = layers.Conv2D(256, (3, 3), activation="relu", padding="same")(x)

    x = layers.Conv2DTranspose(256, (3, 3), strides=(2, 2), padding="same")(x)  # (28, 28)
    x = layers.Conv2D(128, (3, 3), activation="relu", padding="same")(x)

    x = layers.Conv2DTranspose(128, (3, 3), strides=(2, 2), padding="same")(x)  # (56, 56)
    x = layers.Conv2D(64, (3, 3), activation="relu", padding="same")(x)

    x = layers.Conv2DTranspose(64, (3, 3), strides=(2, 2), padding="same")(x)  # (112, 112)
    x = layers.Conv2D(32, (3, 3), activation="relu", padding="same")(x)

    x = layers.Conv2DTranspose(32, (3, 3), strides=(2, 2), padding="same")(x)  # (224, 224)
    x = layers.Conv2D(16, (3, 3), activation="relu", padding="same")(x)

    outputs = layers.Conv2D(1, (1, 1), activation="sigmoid", padding="same")(x)  # (224, 224, 1)

    return keras.Model(inputs, outputs)





In [14]:
try:
    print(resnet_unet.summary())
    print(mobilenet_unet.summary())
    print(efficientnet_unet.summary())
    print(densenet_unet.summary())
except NameError as e:
    print("Error:", e)


Error: name 'resnet_unet' is not defined


In [15]:
# Define U-Net with ResNet50 Backbone
resnet_unet = build_unet_backbone(ResNet50)

# Define U-Net with MobileNetV2 Backbone
mobilenet_unet = build_unet_backbone(MobileNetV2)

# Define U-Net with EfficientNetB0 Backbone
efficientnet_unet = build_unet_backbone(EfficientNetB0)

# Define U-Net with DenseNet121 Backbone
densenet_unet = build_unet_backbone(DenseNet121)


In [16]:
print("U-Net Output Shape:", unet_model.output_shape)
print("ResNet50 U-Net Output Shape:", resnet_unet.output_shape)
print("MobileNetV2 U-Net Output Shape:", mobilenet_unet.output_shape)
print("EfficientNetB0 U-Net Output Shape:", efficientnet_unet.output_shape)
print("DenseNet121 U-Net Output Shape:", densenet_unet.output_shape)


U-Net Output Shape: (None, 224, 224, 1)
ResNet50 U-Net Output Shape: (None, 224, 224, 1)
MobileNetV2 U-Net Output Shape: (None, 224, 224, 1)
EfficientNetB0 U-Net Output Shape: (None, 224, 224, 1)
DenseNet121 U-Net Output Shape: (None, 224, 224, 1)


In [17]:
def compile_and_train(model, name):
    # ✅ Use AdamW instead of Adam for better stability
    optimizer = tf.keras.optimizers.AdamW(learning_rate=1e-3, weight_decay=1e-4)

    model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])

    # ✅ Increase patience for early stopping (to 20 epochs)
    early_stopping = EarlyStopping(monitor="val_loss", patience=20, restore_best_weights=True)

    # ✅ Reduce LR decay sensitivity (factor=0.7, min_lr=1e-7)
    reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.7, patience=5, min_lr=1e-7)

    history = model.fit(
        X_train, Y_train, validation_data=(X_val, Y_val),
        batch_size=32,  # ✅ Increase batch size for better gradient updates
        epochs=100,  # ✅ Allow more epochs
        callbacks=[early_stopping, reduce_lr]
    )

    model.save(f"{name}.h5")
    return history






In [18]:
histories = {
    "U-Net": compile_and_train(unet_model, "unet"),
    "ResNet50 U-Net": compile_and_train(resnet_unet, "resnet_unet"),
    "MobileNet U-Net": compile_and_train(mobilenet_unet, "mobilenet_unet"),
    "EfficientNet U-Net": compile_and_train(efficientnet_unet, "efficientnet_unet"),
    "DenseNet U-Net": compile_and_train(densenet_unet, "densenet_unet"),
}



Epoch 1/100
51/51 ━━━━━━━━━━━━━━━━━━━━ 91s 995ms/step - accuracy: 0.6335 - loss: 0.6473 - val_accuracy: 0.6915 - val_loss: 0.5874 - learning_rate: 0.0010
Epoch 2/100
51/51 ━━━━━━━━━━━━━━━━━━━━ 21s 410ms/step - accuracy: 0.6858 - loss: 0.5888 - val_accuracy: 0.7043 - val_loss: 0.5808 - learning_rate: 0.0010
Epoch 3/100
51/51 ━━━━━━━━━━━━━━━━━━━━ 21s 410ms/step - accuracy: 0.6986 - loss: 0.5783 - val_accuracy: 0.6388 - val_loss: 0.6194 - learning_rate: 0.0010
Epoch 4/100
51/51 ━━━━━━━━━━━━━━━━━━━━ 21s 409ms/step - accuracy: 0.6911 - loss: 0.6329 - val_accuracy: 0.6638 - val_loss: 0.6232 - learning_rate: 0.0010
Epoch 5/100
51/51 ━━━━━━━━━━━━━━━━━━━━ 21s 409ms/step - accuracy: 0.6577 - loss: 0.6135 - val_accuracy: 0.6820 - val_loss: 0.6060 - learning_rate: 0.0010
Epoch 6/100
51/51 ━━━━━━━━━━━━━━━━━━━━ 21s 408ms/step - accuracy: 0.6754 - loss: 0.6169 - val_accuracy: 0.6399 - val_loss: 0.7319 - learning_rate: 0.0010
Epoch 7/100
51/51 ━━━━━━━━━━━━━━━━━━━━ 21s 409ms/step - accuracy: 0.6217 - l

In [19]:
from tensorflow.keras.metrics import MeanIoU
from sklearn.metrics import precision_score, recall_score, accuracy_score, jaccard_score
import numpy as np

def dice_coefficient(y_true, y_pred, smooth=1e-6):
    """
    Compute the Dice coefficient (F1-score for segmentation).
    """
    intersection = np.sum(y_true * y_pred)
    return (2. * intersection + smooth) / (np.sum(y_true) + np.sum(y_pred) + smooth)

def evaluate_model(model, X_test, Y_test, threshold=0.5):
    """
    Evaluate a trained model on the test dataset using multiple metrics.
    
    Parameters:
        model (tf.keras.Model): Trained segmentation model.
        X_test (numpy array): Test images.
        Y_test (numpy array): Ground truth masks.
        threshold (float): Threshold for binary mask conversion.

    Returns:
        dict: Dictionary containing all metric scores.
    """
    # Predict segmentation masks
    Y_pred = model.predict(X_test, batch_size=8)
    
    # Convert probabilities to binary masks (Thresholding)
    Y_pred_binary = (Y_pred > threshold).astype(np.uint8)

    # Flatten arrays for metric calculations
    Y_test_flat = Y_test.flatten()
    Y_pred_flat = Y_pred_binary.flatten()

    # Compute metrics
    accuracy = accuracy_score(Y_test_flat, Y_pred_flat)
    precision = precision_score(Y_test_flat, Y_pred_flat, zero_division=1)
    recall = recall_score(Y_test_flat, Y_pred_flat, zero_division=1)
    jaccard = jaccard_score(Y_test_flat, Y_pred_flat, zero_division=1)
    dice = dice_coefficient(Y_test_flat, Y_pred_flat)

    # Compute IoU (Mean Intersection over Union)
    mean_iou = MeanIoU(num_classes=2)
    mean_iou.update_state(Y_test, Y_pred_binary)
    iou_score = mean_iou.result().numpy()

    # Print and return results
    metrics = {
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "Jaccard Coefficient (IoU)": jaccard,
        "Dice Coefficient": dice,
        "Mean IoU": iou_score
    }

    print(f"\n===== Model Evaluation Results =====")
    for key, value in metrics.items():
        print(f"{key}: {value:.4f}")

    return metrics


In [20]:
models = {
    "U-Net": unet_model,
    "ResNet50 U-Net": resnet_unet,
    "MobileNet U-Net": mobilenet_unet,
    "EfficientNet U-Net": efficientnet_unet,
    "DenseNet U-Net": densenet_unet
}

# Evaluate each model on the test set
model_results = {}
for name, model in models.items():
    print(f"\nEvaluating {name} on Test Set...")
    model_results[name] = evaluate_model(model, X_test, Y_test)



Evaluating U-Net on Test Set...
56/56 ━━━━━━━━━━━━━━━━━━━━ 11s 113ms/step

===== Model Evaluation Results =====
Accuracy: 0.8010
Precision: 0.8311
Recall: 0.8627
Jaccard Coefficient (IoU): 0.7340
Dice Coefficient: 0.8466
Mean IoU: 0.6463

Evaluating ResNet50 U-Net on Test Set...
56/56 ━━━━━━━━━━━━━━━━━━━━ 12s 113ms/step

===== Model Evaluation Results =====
Accuracy: 0.8172
Precision: 0.8038
Recall: 0.9430
Jaccard Coefficient (IoU): 0.7666
Dice Coefficient: 0.8679
Mean IoU: 0.6547

Evaluating MobileNet U-Net on Test Set...
56/56 ━━━━━━━━━━━━━━━━━━━━ 9s 91ms/step

===== Model Evaluation Results =====
Accuracy: 0.6360
Precision: 0.6388
Recall: 0.9857
Jaccard Coefficient (IoU): 0.6329
Dice Coefficient: 0.7752
Mean IoU: 0.3280

Evaluating EfficientNet U-Net on Test Set...
56/56 ━━━━━━━━━━━━━━━━━━━━ 15s 148ms/step

===== Model Evaluation Results =====
Accuracy: 0.7542
Precision: 0.7834
Recall: 0.8485
Jaccard Coefficient (IoU): 0.6872
Dice Coefficient: 0.8146
Mean IoU: 0.5764

Evaluating De